In [6]:
from dataclasses import dataclass, asdict
from os import path, environ

base_dir = "11"

@dataclass
class Config:
    BASE_DIR = base_dir
    
    DB_POOL_RECYCLE:int = 900
    DB_ECGO: bool = True
        
        
@dataclass
class LocalConfig(Config):
    PROJ_RELOAD: bool = True
        
        
def conf():    
    config = dict(local=LocalConfig())
    return config.get(environ.get("API_ENV", "local"))
    
print(asdict(LocalConfig()))
print(type(asdict(LocalConfig())))
print(asdict(conf()))

{'DB_POOL_RECYCLE': 900, 'DB_ECGO': True, 'PROJ_RELOAD': True}
<class 'dict'>
{'DB_POOL_RECYCLE': 900, 'DB_ECGO': True, 'PROJ_RELOAD': True}


In [119]:
import pandas as pd 


df = pd.read_parquet('../parquet/export_ae_db_sql_text_220822.parquet')

sqlparse.parse(df['sql_text'][0])

(<Statement 'SELECT...' at 0x23B9C3BFD80>,)

In [11]:
df.head()

,sql_uid,partition_key,sql_text,total_len,first_token,last_token_len,last_token
0,00FE5BE090A6A5CFE73944A89D695BF4FD27DD4D,220822001,"SELECT /*+ INDEX (""C"" ""PK_CBIDEPART"") */ ""DEPT...",10,SELECT,7,"WHERE ""DEPT_CD""='RM' AND ""BSNS_CD""='11'"
1,012C100B63F7159470F1AC5AD06C3CC474674190,220822001,"select pt_acpt_dte , ...",38,select,3,update_dte desc
2,0501DDB9936D4F0A62BC105830EA85A5A289EF0E,220822001,Begin PG_MOD_DschOrder.Proc_SetDschOrder(in_mo...,5,Begin,1,
3,0700C094781564AAF019CE7306B3BB814714F634,220822001,"SELECT SPC_CD SPC_CD, SPC_NAME SPC_NM FROM SLA...",11,SELECT,1,1
4,082428AC74AA6E780075E1A8C4E2FE4A84BB5FFF,220822001,"select a.med_rcd_iddoctor_note_id, a.RELEA...",15,select,8,"where a.med_rcd_id In(2022123157, 2022123154, ..."


In [105]:
import re 
import sqlparse

def _remove_unnecess_char(df):
    repls = {r'\\t': '', r'\\n': '', r'\\r': '', '\t': '', '\n': '', '\r': ''}
    rep = dict((re.escape(k), v) for k, v in repls.items())
    pattern = re.compile("|".join(rep.keys()))

    df['sql_text'] = df['sql_text'].apply(
        lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x)
    )
    return df
    


In [124]:
def _parse_sql_text(df):
    df['p_sql_text'] = df['sql_text'].apply(
        lambda x: sqlparse.parse(
            x
            # sqlparse.format(x, keyword_case='upper', identifier_case='upper', strip_comments=True)
        )
    )
    return df

def _new_parse_sql_text(df):
    df['p_sql_text'] = pd.Series([list(sqlparse.parse(text)) for text in zip(df['sql_text'])])
    return df

def _set_token_compare_info(df):
    df['total_len'] = df['p_sql_text'].apply(
        lambda x: len(x[0].tokens)
    )
    df['first_token'] = df['p_sql_text'].apply(
        lambda x: str(x[0].tokens[0])
    )
    df['last_token_len'] = df['p_sql_text'].apply(
        lambda x: len(x[0].tokens[-1].tokens) if getattr(x[0].tokens[-1], 'tokens', False) else 1
    )
    df['last_token'] = df['p_sql_text'].apply(
        lambda x: str(x[0].tokens[-1])
    )
    # df['sql_text'] = df[target_c].apply(
    #     self._tokens_flatten
    # )
    return df

def _preprocessing(df):
        
#     df = _remove_unnecess_char(df)
    df = _new_parse_sql_text(df)
#     df = _set_token_compare_info(df)
    return df


In [125]:
df = pd.read_parquet('../parquet/export_ae_db_sql_text_220826.parquet')

%timeit _preprocessing(df[:100])

TypeError: Expected text or file-like object, got <class 'tuple'>

In [86]:

df = pd.read_parquet('../parquet/export_ae_db_sql_text_220826.parquet')

df = _remove_unnecess_char(df)



In [83]:
%timeit df[:100]['sql_text'].str.split("\s+").count()

34.5 ms ± 4.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [145]:
df['sp_sql_text'] = df['sql_text'].str.split("\s+")
df

,sql_uid,partition_key,sql_text,total_len,first_token,last_token_len,last_token,sp_sql_text
0,09594BFBC46AA3C4921EB28E12923A974BF4FEBC,220826001,SELECT /*+ all_rows */ a....,31,SELECT,2,"(SELECT ROWNUM AA , a...","[SELECT, /*+, all_rows, */, a.*, FROM, (SELECT..."
1,0BAF592D6FC06C3F000923C90282EF1AB1F9BB8E,220826001,"select a.med_rcd_id doctor_note_id , a.r...",613,select,22,where a.med_rcd_id = 2023376979 and a.re...,"[select, a.med_rcd_id, doctor_note_id, ,, a.re..."
2,0C8DDFBBA9C92FD9DFD1572EAE887664B2D93FD8,220826001,"SELECT NVL(d.mng_no,' ') AS RUID ...",33,SELECT,1,"-- 재활은 싱글수가만 존재한다. SELECT NVL(d.mng_no,' ') ...","[SELECT, NVL(d.mng_no,', '), AS, RUID, ,, NVL(..."
3,11FB96B37E79A5C18BC5FE696FC6A2F0958C0938,220826001,SELECT m.pt_no ...,115,SELECT,4,"m.shot_dttm DESC, a.exam_grpcd","[SELECT, m.pt_no, PTNO, ,TO_CHAR(m.shot_dttm,'..."
4,12BE3BAA4F387D54464334EF0350FFF556FE3E49,220826001,"select a.med_rcd_id doctor_note_id , a.r...",121,select,22,where a.med_rcd_id = 2022906483 and a.re...,"[select, a.med_rcd_id, doctor_note_id, ,, a.re..."
...,...,...,...,...,...,...,...,...
46920,C8A83E2F3003CDB91F424777D978BFFBC874DD98,220826001,"select med_rcd_id DOCTOR_NOTE_ID, RELEASE_N...",117,select,67,"( select med_rcd_id, RELEASE_NUMBER, item_id, ...","[select, med_rcd_id, DOCTOR_NOTE_ID,, RELEASE_..."
46921,CEF98E49CDFF1D5123316BEC826DE8CB5A153636,220826001,"SELECT '', A.spc_no, TO_CHAR(A.bld_col_date, '...",15,SELECT,23,"A.spc_no, TO_CHAR(A.bld_col_date, 'YYYYMMDD'),...","[SELECT, '',, A.spc_no,, TO_CHAR(A.bld_col_dat..."
46922,DAB8EDE48BA16577B87FA5A9EDB7C53F48A5C3B8,220826001,SELECT m.pt_no ...,115,SELECT,4,"m.shot_dttm DESC, a.exam_grpcd","[SELECT, m.pt_no, PTNO, ,TO_CHAR(m.shot_dttm,'..."
46923,F567A12AA3C17F7BE9A6CD2E8B305BB1A9F5AFEC,220826001,SELECT /*+ ordered use_nl(m) index (M SREXAMDT...,168,SELECT,4,"M.shot_dttm DESC, a.exam_grpcd","[SELECT, /*+, ordered, use_nl(m), index, (M, S..."


In [147]:
df['sp_sql_text_len'] = df['sp_sql_text'].apply(len)

In [148]:
df

,sql_uid,partition_key,sql_text,total_len,first_token,last_token_len,last_token,sp_sql_text,sp_sql_text_len
0,09594BFBC46AA3C4921EB28E12923A974BF4FEBC,220826001,SELECT /*+ all_rows */ a....,31,SELECT,2,"(SELECT ROWNUM AA , a...","[SELECT, /*+, all_rows, */, a.*, FROM, (SELECT...",1468
1,0BAF592D6FC06C3F000923C90282EF1AB1F9BB8E,220826001,"select a.med_rcd_id doctor_note_id , a.r...",613,select,22,where a.med_rcd_id = 2023376979 and a.re...,"[select, a.med_rcd_id, doctor_note_id, ,, a.re...",1123
2,0C8DDFBBA9C92FD9DFD1572EAE887664B2D93FD8,220826001,"SELECT NVL(d.mng_no,' ') AS RUID ...",33,SELECT,1,"-- 재활은 싱글수가만 존재한다. SELECT NVL(d.mng_no,' ') ...","[SELECT, NVL(d.mng_no,', '), AS, RUID, ,, NVL(...",1471
3,11FB96B37E79A5C18BC5FE696FC6A2F0958C0938,220826001,SELECT m.pt_no ...,115,SELECT,4,"m.shot_dttm DESC, a.exam_grpcd","[SELECT, m.pt_no, PTNO, ,TO_CHAR(m.shot_dttm,'...",185
4,12BE3BAA4F387D54464334EF0350FFF556FE3E49,220826001,"select a.med_rcd_id doctor_note_id , a.r...",121,select,22,where a.med_rcd_id = 2022906483 and a.re...,"[select, a.med_rcd_id, doctor_note_id, ,, a.re...",223
...,...,...,...,...,...,...,...,...,...
46920,C8A83E2F3003CDB91F424777D978BFFBC874DD98,220826001,"select med_rcd_id DOCTOR_NOTE_ID, RELEASE_N...",117,select,67,"( select med_rcd_id, RELEASE_NUMBER, item_id, ...","[select, med_rcd_id, DOCTOR_NOTE_ID,, RELEASE_...",1534
46921,CEF98E49CDFF1D5123316BEC826DE8CB5A153636,220826001,"SELECT '', A.spc_no, TO_CHAR(A.bld_col_date, '...",15,SELECT,23,"A.spc_no, TO_CHAR(A.bld_col_date, 'YYYYMMDD'),...","[SELECT, '',, A.spc_no,, TO_CHAR(A.bld_col_dat...",65
46922,DAB8EDE48BA16577B87FA5A9EDB7C53F48A5C3B8,220826001,SELECT m.pt_no ...,115,SELECT,4,"m.shot_dttm DESC, a.exam_grpcd","[SELECT, m.pt_no, PTNO, ,TO_CHAR(m.shot_dttm,'...",187
46923,F567A12AA3C17F7BE9A6CD2E8B305BB1A9F5AFEC,220826001,SELECT /*+ ordered use_nl(m) index (M SREXAMDT...,168,SELECT,4,"M.shot_dttm DESC, a.exam_grpcd","[SELECT, /*+, ordered, use_nl(m), index, (M, S...",118


In [149]:
%timeit df['sp_sql_text'].apply(len)

20.6 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [130]:
%timeit df['sql_text'].str.split("\s+")

12.1 s ± 888 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [131]:
df['sp_sql_text'] = df['sql_text'].str.split("\s+")

In [134]:
df[:5]['sp_sql_text']

0    [SELECT, /*+, all_rows, */, a.*, FROM, (SELECT...
1    [select, a.med_rcd_id, doctor_note_id, ,, a.re...
2    [SELECT, NVL(d.mng_no,', '), AS, RUID, ,, NVL(...
3    [SELECT, m.pt_no, PTNO, ,TO_CHAR(m.shot_dttm,'...
4    [select, a.med_rcd_id, doctor_note_id, ,, a.re...
Name: sp_sql_text, dtype: object

In [137]:
sql = df[:5]['sql_text'][2]

In [138]:
type(sql)

str

In [139]:
p_sql = sqlparse.parse(sql)

In [140]:
len(p_sql[0].tokens)

33

In [141]:
len(sql)

13656

In [142]:
df_sql = df[:5]['sp_sql_text'][2]

In [143]:
type(df_sql)

list

In [144]:
len(df_sql)

1471

In [157]:
df = pd.read_parquet('../split_parquet/export_ae_db_sql_text_220826.parquet')

In [158]:
df

,sql_uid,partition_key,sql_text
0,000CA8FC53C0A1C9D9FFE7E63418B01A4449CD5E,220826001,SELECT /*+ use_nl(a b c d e f) */ ...
1,00175017029C0039DD3DEBE7AA98C67191621551,220826001,SELECT /*+ use_nl(a b c d e f) */ ...
2,001CA4BA06BD8DE3082C019C8A0BDC975EA553C2,220826001,"SELECT '', A.spc_no, TO_CHAR(A.bld_col_date, '..."
3,0025BC92BBBB05CDABE45D83A5E6CC46A83E54C9,220826001,SELECT * FROM ...
4,00399C5F7148682DA7DE9004AD7341AF62110301,220826001,"select a.med_rcd_id doctor_note_id, a.REL..."
...,...,...,...
46920,FF776A73CA6E5E08465149CF3562BE039D6207DB,220826001,"SELECT '', A.spc_no, TO_CHAR(A.bld_col_date, '..."
46921,FF7D1560788CBBBB62A8364F0397C1E78752CE49,220826001,"select a.med_rcd_id DOCTOR_NOTE_ID, a.REL..."
46922,FFA9AC81DBE7BCE24AA6F7C9EDA7FE4D21B157A8,220826001,"select substrb(dump(val,16,0,32),1,120) ep, cn..."
46923,FFB61811987E7A74FC3BA5F89E54B852E08197F0,220826001,"SELECT '', A.spc_no, TO_CHAR(A.bld_col_date, '..."


In [160]:
import numpy as np

x = np.arange(10)

In [163]:
if (x<5).any(): print('11')

11


In [164]:
type(x)

numpy.ndarray